# RAG Application

In [1]:
! pip install pinecone-client==2.2.4 #restart the kernel after executing this cell

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import ConversationChain, HypotheticalDocumentEmbedder, LLMChain
from langchain_community.vectorstores import Qdrant
from langchain.memory import ConversationSummaryMemory
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.chat_models import ChatMlflow

from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool, Tool
from langchain_anthropic import ChatAnthropic
from langchain import PromptTemplate

from domino_data.vectordb import DominoPineconeConfiguration
from ragatouille import RAGPretrainedModel


import os
import pinecone

/home/domino/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Load the embedding model and embeddings.

In [4]:
# Load the embedding model
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/data/{}/model_cache/'.format(os.environ['DOMINO_PROJECT_NAME'])
embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

/home/domino/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
# Initialize Pinecone index
datasource_name = "pinecone-domino-support"
conf = DominoPineconeConfiguration(datasource=datasource_name)
api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

pinecone.init(
    api_key=api_key,
    environment="domino",
    openapi_config=conf,
)

index = pinecone.Index("domino-support")


In [6]:
chat = ChatMlflow(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="Chat-gpt35turbo-se",
)

conversation_openai = ConversationChain(
    llm=chat,
    memory=ConversationSummaryMemory(llm=chat),
    verbose=False
)

# conversation_anthropic = ConversationChain(
#         llm=ChatAnthropic(model='claude-2.1'),
#         memory=ConversationSummaryMemory(llm=ChatAnthropic(model='claude-2.1')),
#         verbose=False
#     )

messages = [
    {"role": "assistant", "content": "How can I help you today?"}
]

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


### Set up HyDE (Hypothetical Document Embeddings)

HyDe promt templates improve the model response by rephrasing the user's question.

In [7]:
# Setup HyDE

hyde_prompt_template = """You are a virtual assistant for Domino and your task is to answer questions related to Domino which includes general information about Domino
"Please answer the user's question below \n 
Question: {question}
Answer:"""
hyde_prompt = PromptTemplate(input_variables=["question"], template=hyde_prompt_template)
hyde_llm_chain = LLMChain(llm=chat, prompt=hyde_prompt)

hyde_embeddings = HypotheticalDocumentEmbedder(
    llm_chain=hyde_llm_chain, base_embeddings=embeddings
)

### Improving the results

#### Re-Ranking with ColBERT

ColBERT is a model that ranks the vectors returned from the Vector DB to improve the relevance of the results.

#### Add a template for the user query

In [8]:
# Get relevant docs through vector DB

SIMILARITY_THRESHOLD = 0.2

# Number of texts to match (may be less if no suitable match)
NUM_TEXT_MATCHES = 5

# Number of texts to return from reranking
NUM_RERANKING_MATCHES = 3

# Create prompt
template = """ You are a virtual assistant for Domino and your task is to answer questions related to Domino which includes general information about Domino.

                Respond in the style of a polite helpful assistant and do not allude that you have looked up the context.

                Do not hallucinate. If you don't find an answer, you can point user to the official website here: https://docs.dominodatalab.com/ . 

                In your response, include the following url links at the end of your response {url_links} and any other relevant URL links that you refered.

                Also, at the end of your response, ask if your response was helpful". 

                Here is some relevant context: {context}"""

# Load the reranking model
colbert = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# Get relevant docs through vector DB
def get_relevant_docs(user_input, num_matches=NUM_TEXT_MATCHES, use_hyde=True):
   
    if use_hyde:
        embedded_query = hyde_embeddings.embed_query(user_input)
    else:
        embedded_query = embeddings.embed_query(user_input)
    
    relevant_docs = index.query(
        namespace='domino-help',
        vector=embedded_query,
        top_k=num_matches,
        include_values=True,
        include_metadata=True
    )

    matches = relevant_docs["matches"]
    filtered_matches = [match for match in matches if match['score'] >= SIMILARITY_THRESHOLD]
    relevant_docs["matches"] = filtered_matches

    return relevant_docs

 
def build_system_prompt(user_input, rerank=True, use_hyde=True):
    
    try:
        relevant_docs = get_relevant_docs(user_input)
    except Exception as e:
        print(f"Failed to get relevant documents: {e}")
        return "", "Failed to get relevant documents"
    
    actual_num_matches = len(relevant_docs["matches"])
    if actual_num_matches == 0:
        print("No matches found in relevant documents.")
        return "", "No matches found in relevant documents"
    
    urls = set([relevant_docs["matches"][i]["metadata"]["source"] for i in range(actual_num_matches)])
    contexts = [relevant_docs["matches"][i]["metadata"]["text"] for i in range(actual_num_matches)]
    
    if rerank and actual_num_matches >= NUM_RERANKING_MATCHES:
        print(actual_num_matches)
        try:
            docs = colbert.rerank(query=user_input, documents=contexts, k=NUM_RERANKING_MATCHES)
        except Exception as e:
            print(f"Failed to rerank documents: {e}")
            return "", "Failed to rerank documents"
        
        try:
            result_indices = [docs[i]["result_index"] for i in range(NUM_RERANKING_MATCHES)]
        except (IndexError, KeyError) as e:
                print(f"Invalid result indices: {e}")
                return "", "Invalid result indices"
        try:    
            contexts = [contexts[index] for index in result_indices]
            urls = [list(urls)[index] for index in result_indices]
        except IndexError as e:
            print(f"Indexing error: {e}")
            return "", "Indexing error"
    
    # Create prompt
    template = """ You are a virtual assistant for Domino Data Lab and your task is to answer questions related to Domino which includes general information about Domino.

                Respond in the style of a polite helpful assistant and do not allude that you have looked up the context.

                Do not hallucinate. If you don't find an answer, you can point user to the official website here: https://docs.dominodatalab.com/. 

                In your response, include the following url links at the end of your response {url_links} and any other relevant URL links that you refered.

                Also, at the end of your response, ask if your response was helpful". 

                Here is some relevant context: {context}"""
 
    prompt_template = PromptTemplate(
        input_variables=["url_links", "context"],
        template=template
    )
    
    system_prompt = prompt_template.format( url_links=urls, context=contexts)
 
    return system_prompt, contexts
 
# Query the Open AI Model
def queryAIModel(user_input, llm_name="openai", use_hyde=True):

    if not user_input:
        return "Please provide an input"
    
    system_prompt = build_system_prompt(user_input)            
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=user_input
        ),
    ]
    if llm_name.lower() == "openai":
        output = conversation_openai.predict(input=messages)
    else:
        output = conversation_anthropic.predict(input=messages)

    return output

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/home/domino/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
# Ask a question ; uncomment to test

# What options do I have for adding code packages to my executions?

user_question = input("Please provide your question here :")
result = queryAIModel(user_question)
result

Please provide your question here : What options do I have for adding code packages to my executions?


Failed to get relevant documents: 404 Client Error: Not Found for url: http://127.0.0.1:8767/endpoints/Chat-gpt35turbo-se/invocations. Response text: {"detail":"Not Found"}


HTTPError: 404 Client Error: Not Found for url: http://127.0.0.1:8767/endpoints/Chat-gpt35turbo-se/invocations. Response text: {"detail":"Not Found"}